In [ ]:
!pip install pulp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 44.1 MB/s eta 0:00:00


#Lembre de botar os arquivos teste no colab, "teste1.txt", "teste2.txt", "teste3.txt", "teste4.txt"

#Classes No e Arvore

In [ ]:
class No:
    def __init__(self,resultado, indice, valor, variavel):
        self.filho_esquerdo = None              # Referência para o filho esquerdo
        self.filho_direito = None               # Referência para o filho direito
        self.resultado = resultado              # Resultado da função objetivo
        self.variavel = variavel                # Lista com valor das variáveis
        self.indice = indice                    # Indica qual variável vai ser adicionada nas restrições especificas
        self.valor = valor                      # Valores 0 ou 1 para o indice
class ArvoreBinaria:
    def __init__(self):
        self.raiz = None
        self.melhor_resultado = float('-inf')  #Melhor resultado
        self.variaveis = None                  #Valores das variaveis para chegar no melhor resultado

    def inserir(self, no):
        if self.raiz is None:
            self.raiz = no
        else:
            self._inserir_recursivo(self.raiz, no)

    #[]
    #[1]
    #[1,0]
    #[1,0,1]

    def _inserir_recursivo(self, atual, no):

        #Vamos inserir à esquerda se o valor do novo nó for 0 e à direita se for 1

        if no.valor[len(atual.valor)] == 0:     #Garante que chegue no Nó do qual herdou as caracteristicas
            if atual.filho_esquerdo is None:
                atual.filho_esquerdo = no
            else:
                self._inserir_recursivo(atual.filho_esquerdo, no)
        else:
            if atual.filho_direito is None:
                atual.filho_direito = no
            else:
                self._inserir_recursivo(atual.filho_direito, no)

    def resultado_final(self):
      print(f'Melhor resultado: {self.melhor_resultado}')
      for i in range(len(self.variaveis)):
        print(f"x{i+1} = {self.variaveis[i]}")

#Determinar o elemento mais próximo de 0.5 que não é inteiro

In [ ]:
def numero_proximo_05(numeros):

    # Filtra os números que não são inteiros
    nao_inteiros = [num for num in numeros if num % 1 != 0]

    # Se não houver números não inteiros, retorna None
    if not nao_inteiros:
        return None

    # Encontra o número mais próximo de 0.5
    mais_proximo = min(nao_inteiros, key=lambda x: abs(x - 0.5))

    return mais_proximo

#Codigo para capturar os dados do problema

In [ ]:
from pulp import LpProblem, LpVariable, lpSum, LpStatus, LpMaximize, value #JP
#leitura do arquivo
def coletar_informacoes(texto):
  with open(texto,"r") as teste:
    linha_variavel_restricao = teste.readlines()


  #Tentar tirar os espaços em branco e o "\n", e transformar as strings em numeros inteiros
  resultado = []
  for item in linha_variavel_restricao:
      # Remove os espaços em branco e a nova linha

      item = item.strip()

      if item:  # Verifica se a string não está vazia
          # Divide a string em números, converte para inteiro e adiciona à lista resultado
          numeros = list(map(int, item.split()))
          resultado.append(numeros)



  #Como os arquivos sao padronizados dessa forma, os indices serão os mesmos
  variaveis,quant_restricoes = resultado[0][0],resultado[0][1] #Identifica a quantidade de variáveis e quantidade de restrições
  resultado.remove(resultado[0])  #Remove essa linha para que facilite na hora de buscar a funcao_objetivo
  funcao_objetivo = resultado[0] #Pega a FO
  restricoes = resultado[1:] #O restante é restrição



  return variaveis,quant_restricoes,funcao_objetivo,restricoes




#Branch_and_Bound

In [ ]:
from pulp import LpProblem, LpMaximize, LpVariable, LpStatus, value

def Branch_and_Bound(Arvore,indice,valor, variaveis, quant_restricoes, funcao_objetivo, restricoes):

    if(len(indice) != len(valor)): #Caso venham com tamanhos diferentes
      return

    model = LpProblem("Problema", LpMaximize)

    # Criando as variáveis de decisão
    x = [LpVariable(f"x{i+1}", lowBound=0, upBound=1, cat='Continuous') for i in range(variaveis)]

    # Definindo a função objetivo
    model += sum(funcao_objetivo[i] * x[i] for i in range(variaveis)), "Maximizar"

    # Adicionando restrições tiradas desde o inicio
    for i in range(len(restricoes)):
        model += sum(restricoes[i][j] * x[j] for j in range(variaveis))  <= restricoes[i][variaveis]

    #Restrições especificas
    if(len(indice)):
        for i in range(len(indice)):
            model += x[indice[i]] == valor[i], f"Restrição_{i}"


    # Resolvendo o modelo
    model.solve()

    #Tentar cortar por inviabilidade
    if LpStatus[model.status] in ('Infeasible','Unbounded','Undefined'):
        return

    #Tentar cortar por limitante
    if(Arvore.melhor_resultado > value(model.objective)):
      return

    #Vamos colocar os resultados
    # lista_resultados = [round(var.varValue,2) for var in x]
    lista_resultados = [var.varValue for var in x]

    Arvore.inserir(No(value(model.objective),indice,valor,lista_resultados)) #Inserir na árvore o nó
    # Verificar se todos os resultados são inteiros
    t = numero_proximo_05(lista_resultados)
    if t is  None:
        Arvore.melhor_resultado = value(model.objective)
        Arvore.variaveis = lista_resultados
        return #vai cortar por integralidade

    #Pega o indice do valor fracionário mais perto de 0.5
    indix = lista_resultados.index(t)

    indices = indice + [indix] #As restrições do pai + a nova restrição do filho


    valores1, valores2 = valor + [1] , valor + [0] # Os possiveis valores que a variável fracionária pode assumir.


    #Chama a função recursivamente
    Branch_and_Bound(Arvore, indices, valores2, variaveis, quant_restricoes, funcao_objetivo, restricoes)
    Branch_and_Bound(Arvore, indices,valores1, variaveis, quant_restricoes, funcao_objetivo, restricoes)




In [ ]:
def problema(variaveis, quant_restricoes, funcao_objetivo, restricoes):
  Arvore = ArvoreBinaria()
  Branch_and_Bound(Arvore,[],[], variaveis, quant_restricoes, funcao_objetivo, restricoes)
  Arvore.resultado_final()

#Inicalização automática

In [ ]:
 #Lembre de adicionar os arquivos testes ao colab com esses nomes.
lista_de_testes = ["teste1.txt","teste2.txt","teste3.txt","teste4.txt"]
for i in lista_de_testes:
  variaveis, quant_restricoes, funcao_objetivo, restricoes = coletar_informacoes(i)
  problema(variaveis, quant_restricoes, funcao_objetivo, restricoes)

Melhor resultado: 20.0
x1 = 0.0
x2 = 1.0
x3 = 0.0
x4 = 0.0
x5 = 1.0
x6 = 0.0
x7 = 0.0
Melhor resultado: 24.0
x1 = 0.0
x2 = 0.0
x3 = 0.0
x4 = 0.0
x5 = 0.0
x6 = 1.0
x7 = 1.0
x8 = 0.0
x9 = 1.0
Melhor resultado: 19.0
x1 = 0.0
x2 = 0.0
x3 = 1.0
x4 = 0.0
x5 = 0.0
x6 = 0.0
x7 = 1.0
x8 = 0.0
x9 = 0.0
Melhor resultado: 10.0
x1 = 0.0
x2 = 0.0
x3 = 1.0
x4 = 0.0
x5 = 0.0
x6 = 0.0
x7 = 0.0
x8 = 0.0
x9 = 0.0
